In [1]:
# Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
# Checking for device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
# Transforms
transformer = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), #convert 0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1, 1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [5]:
# DataLoader
# Path for the training and tresting directory
train_path = '/home/sadam/Aletheia-AI/Aletheia-AI Developments/Pytorch_Training/intel_image_dataset_training/scene_detection/seg_train/seg_train'
test_path = '/home/sadam/Aletheia-AI/Aletheia-AI Developments/Pytorch_Training/intel_image_dataset_training/scene_detection/seg_test/seg_test'

train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform=transformer),
    batch_size=256, shuffle=True
)
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform=transformer),
    batch_size=256, shuffle=True
)

In [6]:
# Categories
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [7]:
print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [10]:
# CNN Network

class ConvNet(nn.Module):
    def __init__(self, num_classes=6):
        super(ConvNet, self).__init__()
        # Output size after convolution filter
        # ((w-f+2P)/s)+1
        # Input shape = (256, 3, 150, 150)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 12, 150, 150)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        # Shape = (256, 12, 150, 150)
        
        self.relu1 = nn.ReLU()
        # Shape = (256, 12, 150, 150)
        
        self.pool = nn.MaxPool2d(kernel_size=2)
        # Reduce the image size be factor 2
        # Shape = (256, 12, 75, 75)
        
        
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 20, 75, 75)        
        self.relu2 = nn.ReLU()
        # Shape = (256, 20, 75, 75)
        
        
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 32, 75, 75)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        # Shape = (256, 32, 75, 75)
        self.relu3 = nn.ReLU()
        # Shape = (256, 32, 75, 75)
        
        self.fc = nn.Linear(in_features=75 * 75 * 32, out_features=num_classes)
        
        
    # Feed Forward function
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        
        output = self.pool(output)
        
        output = self.conv2(output)
        output = self.relu2(output)
        
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        
        # Above output will be in mstrix form, with shape (256, 32, 75, 75)
        
        output = output.view(-1, 32 * 75 * 75)
        
        output = self.fc(output)
        return output
        
        

In [11]:
model = ConvNet(num_classes=6).to(device)